In [1]:
# !pip install pennylane nftopt

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.6/1.6 MB 9.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.2/2.2 MB 16.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 kB 5.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.1/18.1 MB 32.2 MB/s eta 0:00:00


In [ ]:
# import numpy as np
# arr = []
# for i in range(256):
#   arr.append(1/256)

# arr = np.array(arr)
# print(arr)

In [24]:
import numpy as np
arr = np.random.rand(1,256)
print(arr)

[[0.64494975 0.9983475  0.84657812 0.48452119 0.39084177 0.30541591
  0.91006393 0.45316097 0.70092859 0.19016603 0.29561208 0.38844691
  0.45628678 0.24711698 0.18978742 0.90983018 0.98569438 0.00871219
  0.52173491 0.35993281 0.22049975 0.07259503 0.17977841 0.46817033
  0.26352262 0.61251969 0.4759611  0.46442821 0.97946321 0.90326843
  0.77335068 0.22932426 0.2203521  0.97781107 0.0693043  0.09389387
  0.82638321 0.68755247 0.90726111 0.80151695 0.41616821 0.20575367
  0.83415092 0.68011979 0.78804395 0.29994948 0.0768441  0.09002431
  0.84140926 0.85801449 0.45594483 0.02738024 0.96942872 0.59754626
  0.46749911 0.27152295 0.42427163 0.01024485 0.46554149 0.58661386
  0.31539874 0.43413329 0.84914824 0.10174096 0.59805639 0.58869913
  0.85822889 0.27059124 0.35960685 0.73457653 0.79778826 0.3593671
  0.92332486 0.64134032 0.33053361 0.4124786  0.67787779 0.28153181
  0.19187443 0.88432798 0.44442116 0.21307866 0.97256903 0.72924828
  0.3672578  0.17404328 0.34699167 0.03572695 0.9

In [ ]:
np.unique(arr)

In [26]:
summer = np.sum(arr)
print(summer)
normalized_X = arr/summer
print(normalized_X)

128.26185815919507
[[5.02838303e-03 7.78366627e-03 6.60038871e-03 3.77759376e-03
  3.04721746e-03 2.38119041e-03 7.09535903e-03 3.53309219e-03
  5.46482481e-03 1.48263896e-03 2.30475437e-03 3.02854578e-03
  3.55746272e-03 1.92665992e-03 1.47968711e-03 7.09353655e-03
  7.68501561e-03 6.79250505e-05 4.06773236e-03 2.80623418e-03
  1.71913737e-03 5.65990770e-04 1.40165134e-03 3.65011339e-03
  2.05456730e-03 4.77554044e-03 3.71085457e-03 3.62093779e-03
  7.63643400e-03 7.04237758e-03 6.02946729e-03 1.78793804e-03
  1.71798616e-03 7.62355298e-03 5.40334483e-04 7.32048223e-04
  6.44293812e-03 5.36053725e-03 7.07350664e-03 6.24906701e-03
  3.24467627e-03 1.60416881e-03 6.50349938e-03 5.30258801e-03
  6.14402413e-03 2.33857113e-03 5.99118870e-04 7.01879045e-04
  6.56008942e-03 6.68955295e-03 3.55479667e-03 2.13471424e-04
  7.55819954e-03 4.65879931e-03 3.64488023e-03 2.11694227e-03
  3.30785502e-03 7.98744686e-05 3.62961756e-03 4.57356437e-03
  2.45902208e-03 3.38474194e-03 6.62042679e-03 7.93

In [27]:
#defining parameters
n_qubits = 8
depth = 4

In [28]:
import pennylane as qml
from pennylane import numpy as np
from pennylane.optimize import NesterovMomentumOptimizer
from scipy import optimize
from nftopt import nakanishi_fujii_todo as nftmethod

In [29]:
dev = qml.device("default.qubit",wires=n_qubits)
dev

<default.qubit device (wires=8) at 0x7afe23508fd0>

In [30]:
def ansatz_layer(layer_weights, depth, n_qubits):
    for dep in range(depth):
      for wire in range(n_qubits):
        qml.RY(layer_weights[wire+(n_qubits*dep)], wires=wire)

      qml.CZ(wires=[0,1])
      qml.CZ(wires=[1,2])
      qml.CZ(wires=[2,3])
      qml.CZ(wires=[3,4])
      qml.CZ(wires=[4,5])
      qml.CZ(wires=[5,6])
      qml.CZ(wires=[6,7])
      qml.CZ(wires=[7,0])

In [31]:
def accuracy(labels, predictions):
  state0 = qml.math.dm_from_state_vector(labels)
  state1 = qml.math.dm_from_state_vector(predictions)
  return qml.math.fidelity(state0, state1)

In [32]:
@qml.qnode(dev,diff_method="backprop")
def circuit(weights):
  # since depth 4 was used
  ansatz_layer(weights,depth=depth, n_qubits=n_qubits)
  # qml.state() applies Ua to Ini State, which gives Appro_a
  return qml.state()

In [33]:
def variational_classifier(weights, x):
    # weights are thetas
    return np.real(circuit(weights))

In [36]:
# x = np.array([0.083379922415887,0.049577998492674,0.989090451339352,0.049577998492674,0.049577998492674,0.049577998492674,0.049577998492674,0.049577998492674])
# x = np.array([0.31622776601683794,0.31622776601683794,0.31622776601683794,0.4472135954999579,0.31622776601683794,0.31622776601683794,0.31622776601683794,0.4472135954999579])
temp = []
for i in range(len(normalized_X[0])):
  temp.append(np.sqrt(normalized_X[0][i]))
x = np.array(temp)
print(x)

[0.07091109 0.08822509 0.08124278 0.06146213 0.05520161 0.04879744
 0.08423395 0.05943982 0.07392445 0.03850505 0.04800786 0.05503222
 0.05964447 0.04389373 0.0384667  0.08422314 0.08766422 0.00824167
 0.06377878 0.0529739  0.04146248 0.02379056 0.03743863 0.06041617
 0.04532734 0.06910529 0.06091678 0.06017423 0.08738669 0.08391887
 0.07764964 0.04228402 0.0414486  0.08731296 0.0232451  0.02705639
 0.08026791 0.07321569 0.08410414 0.07905104 0.05696206 0.04005208
 0.08064428 0.07281887 0.07838383 0.04835878 0.0244769  0.026493
 0.08099438 0.08178969 0.05962212 0.01461066 0.08693791 0.0682554
 0.06037284 0.04601024 0.05751396 0.00893725 0.06024631 0.06762813
 0.04958853 0.05817854 0.08136601 0.02816431 0.06828453 0.06774823
 0.08179991 0.04593123 0.05294991 0.07567802 0.07886695 0.05293225
 0.08484544 0.07071239 0.05076438 0.05670899 0.07269875 0.04685058
 0.03867762 0.08303437 0.05886384 0.04075878 0.0870786  0.07540306
 0.05351022 0.03683663 0.05201286 0.01668973 0.08464499 0.0256993

In [37]:
sum = 0
for i in range(len(x)):
  sum += x[i]**2
print(sum)

1.0


In [38]:
def F1_loss(a,Appro_a):
  #   Define the cost function
  F1=(a[0]**2)*np.log2((Appro_a[0]**2)/(a[0]**2))+(a[1]**2)*np.log2((Appro_a[1]**2)/(a[1]**2))+(a[2]**2)*np.log2((Appro_a[2]**2)/(a[2]**2))
  F2=(a[3]**2)*np.log2((Appro_a[3]**2)/(a[3]**2))+(a[4]**2)*np.log2((Appro_a[4]**2)/(a[4]**2))+(a[5]**2)*np.log2((Appro_a[5]**2)/(a[5]**2))
  F3=(a[6]**2)*np.log2((Appro_a[6]**2)/(a[6]**2))+(a[7]**2)*np.log2((Appro_a[7]**2)/(a[7]**2))

  #   Prepare the Bell state
  Bell_State=np.ones((8,1))/np.linalg.norm(np.ones((8,1)))
  #   The sum of vector a
  Sum_a=np.sum(a)
  sqrt_D = np.linalg.norm(np.ones((8,1)))
  UaFunction=np.abs(Sum_a -  (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F1+F2+F3)
  return np.real(UaFunction)

In [39]:
def F1_loss(a,Appro_a):
  #   Define the cost function
  F1 = 0.
  for i in range(256):
    F1 += (a[i]**2)*np.log2((Appro_a[i]**2)/(a[i]**2))

  # F1=(a[0]**2)*np.log2((Appro_a[0]**2)/(a[0]**2))+(a[1]**2)*np.log2((Appro_a[1]**2)/(a[1]**2))+(a[2]**2)*np.log2((Appro_a[2]**2)/(a[2]**2))
  # F2=(a[3]**2)*np.log2((Appro_a[3]**2)/(a[3]**2))+(a[4]**2)*np.log2((Appro_a[4]**2)/(a[4]**2))+(a[5]**2)*np.log2((Appro_a[5]**2)/(a[5]**2))
  # F3=(a[6]**2)*np.log2((Appro_a[6]**2)/(a[6]**2))+(a[7]**2)*np.log2((Appro_a[7]**2)/(a[7]**2))

  #   Prepare the Bell state
  states = 2**n_qubits
  Bell_State=np.ones((states,1))/np.linalg.norm(np.ones((states,1)))
  #   The sum of vector a
  Sum_a=np.sum(a)
  sqrt_D = np.linalg.norm(np.ones((states,1)))
  UaFunction=np.abs(Sum_a -  (sqrt_D * np.matmul(np.transpose(Bell_State), Appro_a)))-(F1)
  return np.real(UaFunction)

In [40]:
a= x

In [41]:
def cost(weights, X=x):
    # X is our preparation state
    Appro_a = variational_classifier(weights, X)
    return F1_loss(X, Appro_a)

In [42]:
weights = 4*np.pi*np.random.rand(n_qubits * depth)
result= optimize.minimize(cost,weights, method=nftmethod, options={'maxfev':2000})


In [43]:
result.x

array([ 9.70630057,  3.46629087, 12.28670403, 11.37207788,  6.2602373 ,
       10.89261346,  6.32556393,  2.88526113,  0.09146338,  4.72497145,
       -0.12093828,  7.47217785,  6.07368169,  4.79856298,  3.15821738,
        4.64687359,  7.86183387,  9.11753195, 11.03911367,  4.67607412,
        4.5331367 , 10.97302782,  1.57648483,  2.90052914, 12.14408298,
        6.2649404 , 12.26415949,  9.4293738 ,  6.27555625,  6.25728562,
       12.48253204,  9.41063055])

In [44]:
predictions=variational_classifier(result.x,a)
acc = accuracy(x, predictions)
print(predictions.numpy(), x.numpy())
print(acc)

[0.0745552  0.06353757 0.06870998 0.0645037  0.0698083  0.06598289
 0.07516542 0.06346264 0.068519   0.05936077 0.06787638 0.06222937
 0.06656056 0.06149868 0.06661987 0.05725715 0.06715338 0.05803576
 0.06216926 0.05873961 0.06149631 0.05858218 0.06600885 0.05655468
 0.06650421 0.05815481 0.06565148 0.06121737 0.0660918  0.06207391
 0.0664797  0.05758951 0.0656142  0.05523438 0.06073708 0.05584714
 0.06181537 0.05729217 0.06628783 0.0553469  0.06673444 0.05692649
 0.06581421 0.05996547 0.06456828 0.05923406 0.06494371 0.05489656
 0.07157659 0.05962503 0.06600482 0.06063539 0.06529872 0.06043043
 0.07044042 0.05810606 0.06374386 0.05391912 0.06327483 0.05650931
 0.0637883  0.05745987 0.06389324 0.05360052 0.06148611 0.05825776
 0.05705494 0.058548   0.0580764  0.06043462 0.06229265 0.05816715
 0.06300487 0.06049064 0.06231672 0.06330621 0.06096221 0.06273858
 0.06130109 0.05795774 0.06840414 0.0641418  0.06323981 0.06484159
 0.06238407 0.06483418 0.06732039 0.0621081  0.06045616 0.0576